# RoI and LoI analysis (2D experiments only)

In [1]:
import pickle5
import statistics

In [2]:
# get required information from user
DataPath = "../" # the folder containing the required files
OptFN = f"{DataPath}options_full.ini"

In [3]:
#### FUNCTIONS ####
# get run parameters from options file	
def get_Options(OptFN, OptionsDic, AllowedPars):
    try:
        optionsfile=open(OptFN, "r")
        for line in optionsfile :
            if line[:1] != "#" and len(line.strip("\n".strip(""))) > 1 :
                splitted_line = line.split(":")
                key=splitted_line[0].split(" ")[0]
                value=splitted_line[1].strip("\n").split(" ")[0]
                if key in AllowedPars :
                    OptionsDic[key] = value 
    except:
        print('Error reading %s file !' % (OptFN))

    return OptionsDic

#retrieve list of lists based on 2 separators
def double_split(Text, sep1, sep2):
    # further parse options
    WholeL = []
    MainL = Text.split(sep1)
    for text in MainL:
        WholeL.append(text.strip().split(sep2))

    return WholeL

In [4]:
## load required data
# the options
OptionsDic = {}
AllowedPars = ["Partners"]
OptionsDic = get_Options(OptFN, OptionsDic, AllowedPars)
ComplexesL = double_split(OptionsDic["Partners"], ";", "~")[0]
# the AAvarsDic
AAvarsDic = {}
GlobalAAvarsDic = {}
with open(r"{}SR_AAvars_{}_vs_{}.pkl".format(DataPath,ComplexesL[0], ComplexesL[1]), "rb") as input_file:
    GlobalAAvarsDic = pickle5.load(input_file)
    
## Drop irrelevant information
for complex in GlobalAAvarsDic.keys():
    AAvarsDic = GlobalAAvarsDic[complex]
    for key in AAvarsDic.keys():
        del AAvarsDic[key]['IndBCData']
    
    print("{} single mutant combinations found in SR_AAvars dictionary for complex partners {} (A) and {} (B) !".format(len(AAvarsDic.keys()), ComplexesL[0], ComplexesL[1]))

10956 single mutant combinations found in SR_AAvars dictionary for complex partners VN1551_VHH2 (A) and VN1554_TNFa-2 (B) !


In [5]:
# RoI analysis I
RoIDic = {}
AoBxWT_MaxNormEnrich = 0.7 # maximum allowed enrichment of a single mutant partner against le WT
WT_MinDeltaMax = 0.2 # the minimum difference between a combination of single mutant partners and their combinations (each) with WT

for key in AAvarsDic.keys():
    if int(AAvarsDic[key]['Stats'][2]) >= 6  :
        if AAvarsDic[key]['Stats'][1] <= AAvarsDic[key]['Stats'][0] * 0.7 :
            split_key = key.split(":")
            Partner_A = split_key[0]
            Partner_B = split_key[1]
            if (Partner_A != 'WT') and (Partner_B != 'WT')  and ( f"{Partner_A}:WT" in AAvarsDic.keys() ) and ( f"WT:{Partner_B}" in AAvarsDic.keys() ):
                AxWTb = AAvarsDic[f"{Partner_A}:WT"]['Stats'][0]
                BxWTa = AAvarsDic[f"WT:{Partner_B}"]['Stats'][0]
                if (AxWTb <= AoBxWT_MaxNormEnrich) or (BxWTa <= AoBxWT_MaxNormEnrich):
                    AxB = AAvarsDic[key]['Stats'][0]
                    DeltaA = AxB - AxWTb 
                    DeltaB = AxB - BxWTa
                    DeltaABL = [ DeltaA, DeltaB]
                    DeltaMax = max(DeltaABL)
                    DeltaMean = statistics.mean(DeltaABL)
                    AxWTname = f"{Partner_A}:WT"
                    BxWTname = f"WT:{Partner_B}"
                    AxWT_BCs =  AAvarsDic[AxWTname]['Stats'][2]
                    BxWT_BCs =  AAvarsDic[BxWTname]['Stats'][2]
                    AxB_BCs = AAvarsDic[key]['Stats'][2]
                    if DeltaMax >= WT_MinDeltaMax:
                        RoIDic[key] = [AxWTb, BxWTa, AxB, DeltaMean, DeltaMax, AxWT_BCs, BxWT_BCs, AxB_BCs ]

sorted_RoIDic = sorted(RoIDic.items(), key=lambda item: item[1][-1], reverse=True)
for n in range(len(sorted_RoIDic)):
    print(f"{sorted_RoIDic[n][0]} => recovery= {sorted_RoIDic[n][1][4]:.3}, AxWt= {sorted_RoIDic[n][1][0]:.3} ({sorted_RoIDic[n][1][5]}), BxWt= {sorted_RoIDic[n][1][1]:.3} ({sorted_RoIDic[n][1][6]}), AxB= {sorted_RoIDic[n][1][2]:.3} ({sorted_RoIDic[n][1][7]})")

W33R:N92D => recovery= 0.408, AxWt= 0.123 (133), BxWt= 0.935 (145), AxB= 0.532 (13)
F103A:E146S => recovery= 0.273, AxWt= 1.05 (230), BxWt= 0.631 (115), AxB= 0.904 (12)
Y32E:E146S => recovery= 0.214, AxWt= 1.12 (157), BxWt= 0.631 (115), AxB= 0.845 (9)
S101E:E146S => recovery= 0.237, AxWt= 1.1 (160), BxWt= 0.631 (115), AxB= 0.868 (8)
S101T:E146S => recovery= 0.224, AxWt= 1.14 (101), BxWt= 0.631 (115), AxB= 0.855 (7)
S101T:T89L => recovery= 0.322, AxWt= 1.14 (101), BxWt= 0.697 (68), AxB= 1.02 (6)


In [6]:
# RoI analysis II
RoIDic = {}
AoBxWT_MinNormEnrich = 0.9 # minimum allowed enrichment of a single mutant partner against le WT
WT_MinDeltaMax = 0.3 # the minimum difference between a combination of single mutant partners and their combinations (each) with WT

for key in AAvarsDic.keys():
    if int(AAvarsDic[key]['Stats'][2]) >= 6  :
        if AAvarsDic[key]['Stats'][1] <= AAvarsDic[key]['Stats'][0] *0.7 :
            split_key = key.split(":")
            Partner_A = split_key[0]
            Partner_B = split_key[1]
            if (Partner_A != 'WT') and (Partner_B != 'WT')  and ( "{}:WT".format(Partner_A) in AAvarsDic.keys() ) and ( "WT:{}".format(Partner_B) in AAvarsDic.keys() ):
                AxWTb = AAvarsDic["{}:WT".format(Partner_A)]['Stats'][0]
                BxWTa = AAvarsDic["WT:{}".format(Partner_B)]['Stats'][0]
                if (AxWTb >= AoBxWT_MinNormEnrich) and (BxWTa >= AoBxWT_MinNormEnrich):
                    AxB = AAvarsDic[key]['Stats'][0]
                    DeltaA = AxB - AxWTb 
                    DeltaB = AxB - BxWTa
                    DeltaABL = [ DeltaA, DeltaB]
                    DeltaMax = max(DeltaABL)
                    DeltaMean = statistics.mean(DeltaABL)
                    AxWTname = f"{Partner_A}:WT"
                    BxWTname = f"WT:{Partner_B}"
                    AxWT_BCs =  AAvarsDic[AxWTname]['Stats'][2]
                    BxWT_BCs =  AAvarsDic[BxWTname]['Stats'][2]
                    AxB_BCs = AAvarsDic[key]['Stats'][2]
                    if DeltaMax >= WT_MinDeltaMax:
                        RoIDic[key] = [AxWTb, BxWTa, AxB, DeltaMean, DeltaMax, AxWT_BCs, BxWT_BCs, AxB_BCs ]


sorted_RoIDic = sorted(RoIDic.items(), key=lambda item: item[1][-1], reverse=True)
for n in range(len(sorted_RoIDic)):
    print(f"{sorted_RoIDic[n][0]} => recovery= {sorted_RoIDic[n][1][4]:.3}, AxWt= {sorted_RoIDic[n][1][0]:.3} ({sorted_RoIDic[n][1][5]}), BxWt= {sorted_RoIDic[n][1][1]:.3} ({sorted_RoIDic[n][1][6]}), AxB= {sorted_RoIDic[n][1][2]:.3} ({sorted_RoIDic[n][1][7]})")

S101F:S147F => recovery= 0.523, AxWt= 1.13 (97), BxWt= 0.959 (87), AxB= 1.48 (8)
Y32E:S147F => recovery= 0.38, AxWt= 1.12 (157), BxWt= 0.959 (87), AxB= 1.34 (7)
N54S:Q88N => recovery= 0.378, AxWt= 0.985 (83), BxWt= 1.1 (112), AxB= 1.36 (6)


In [7]:
# LoI analysis I
LoIDic = {}
AoBxWT_MinNormEnrich = 0.9 # minimum allowed enrichment of a single mutant partner against le WT
WT_MinDeltaMin = -0.25 # the minimum difference between a combination of single mutant partners and their combinations (each) with WT

for key in AAvarsDic.keys():
    if int(AAvarsDic[key]['Stats'][2]) >= 6  :
        if AAvarsDic[key]['Stats'][1] <= AAvarsDic[key]['Stats'][0] *0.7 :
            split_key = key.split(":")
            Partner_A = split_key[0]
            Partner_B = split_key[1]
            if (Partner_A != 'WT') and (Partner_B != 'WT')  and ( "{}:WT".format(Partner_A) in AAvarsDic.keys() ) and ( "WT:{}".format(Partner_B) in AAvarsDic.keys() ):
                AxWTb = AAvarsDic["{}:WT".format(Partner_A)]['Stats'][0]
                BxWTa = AAvarsDic["WT:{}".format(Partner_B)]['Stats'][0]
                if (AxWTb >= AoBxWT_MinNormEnrich) and (BxWTa >= AoBxWT_MinNormEnrich):
                    AxB = AAvarsDic[key]['Stats'][0]
                    DeltaA = AxB - AxWTb 
                    DeltaB = AxB - BxWTa
                    DeltaABL = [ DeltaA, DeltaB]
                    DeltaMin = min(DeltaABL)
                    DeltaMean = statistics.mean(DeltaABL)
                    AxWTname = f"{Partner_A}:WT"
                    BxWTname = f"WT:{Partner_B}"
                    AxWT_BCs =  AAvarsDic[AxWTname]['Stats'][2]
                    BxWT_BCs =  AAvarsDic[BxWTname]['Stats'][2]
                    AxB_BCs = AAvarsDic[key]['Stats'][2]                    
                    if DeltaMin <= WT_MinDeltaMin:
                        LoIDic[key] = [AxWTb, BxWTa, AxB, DeltaMean, DeltaMin, AxWT_BCs, BxWT_BCs, AxB_BCs ]
                        

sorted_LoIDic = sorted(LoIDic.items(), key=lambda item: item[1][-1], reverse=False)
for n in range(len(sorted_LoIDic)):
    print(f"{sorted_LoIDic[n][0]} => loss= {sorted_LoIDic[n][1][4]:.3}, AxWt= {sorted_LoIDic[n][1][0]:.3} ({sorted_LoIDic[n][1][5]}), BxWt= {sorted_LoIDic[n][1][1]:.3} ({sorted_LoIDic[n][1][6]}), AxB= {sorted_LoIDic[n][1][2]:.3} ({sorted_LoIDic[n][1][7]}) ")

F103R:T77E => loss= -0.254, AxWt= 0.964 (86), BxWt= 0.936 (113), AxB= 0.709 (6) 
R98G:S81T => loss= -0.552, AxWt= 0.919 (151), BxWt= 1.01 (71), AxB= 0.459 (6) 
S101F:T77E => loss= -0.289, AxWt= 1.13 (97), BxWt= 0.936 (113), AxB= 0.846 (6) 
N54S:N92D => loss= -0.334, AxWt= 0.985 (83), BxWt= 0.935 (145), AxB= 0.651 (6) 
R98L:N92D => loss= -0.29, AxWt= 1.33 (69), BxWt= 0.935 (145), AxB= 1.04 (6) 
S101T:N92A => loss= -0.329, AxWt= 1.14 (101), BxWt= 0.994 (82), AxB= 0.811 (6) 
R98E:N92D => loss= -0.332, AxWt= 0.96 (64), BxWt= 0.935 (145), AxB= 0.629 (7) 
R98G:S95T => loss= -0.277, AxWt= 0.919 (151), BxWt= 1.01 (99), AxB= 0.733 (8) 
L56Q:N92D => loss= -0.295, AxWt= 0.969 (108), BxWt= 0.935 (145), AxB= 0.674 (9) 
F103A:S147G => loss= -0.264, AxWt= 1.05 (230), BxWt= 1.04 (53), AxB= 0.787 (9) 
R98G:T77E => loss= -0.373, AxWt= 0.919 (151), BxWt= 0.936 (113), AxB= 0.563 (10) 
L56F:N92D => loss= -0.279, AxWt= 0.942 (104), BxWt= 0.935 (145), AxB= 0.664 (13) 
R98G:N92D => loss= -0.299, AxWt= 0.919 (